In [ ]:
import os
import torch
from threading import Thread
from transformers import AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, AutoModel

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
MODEL_PATH = os.environ.get('MODEL_PATH', '/data/zhenyu/LLM_Model/GLM_4/GLM4_Chat/')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)

model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    # attn_implementation="flash_attention_2", # Use Flash Attention
    # torch_dtype=torch.bfloat16, #using flash-attn must use bfloat16 or float16
    device_map="auto").eval()

In [ ]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(messages):
    stop = StopOnTokens()
    model_inputs = tokenizer.apply_chat_template(messages,
                                                 add_generation_prompt=True,
                                                 tokenize=True,
                                                 return_tensors="pt").to(next(model.parameters()).device)
    streamer = TextIteratorStreamer(tokenizer, timeout=60, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 512,
        "do_sample": False,
        "top_p": 1,
        "temperature": 0.5,
        "stopping_criteria": StoppingCriteriaList([stop]),
        "repetition_penalty": 1,
        "eos_token_id": model.config.eos_token_id,
    }
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    result = ''
    for new_token in streamer:
        result += new_token
        print(new_token, end='')
    return result


In [ ]:
def create_tot_prompt(cac_value, ctai_value, nodule_size):
    return f"""
#肺结节风险评估

## 1. 初始化

请根据以下信息评估肺结节风险并给出建议：
- CAC值: {cac_value}
- CTAI值: {ctai_value}
- 结节大小: {nodule_size} mm

## 2. 风险评估

请按以下步骤进行思考：

1. 评估CAC值
    如果CAC > 9，考虑高风险
    如果3 ≤ CAC ≤ 9，考虑中高风险
    如果CAC < 3，考虑低风险
2. 评估CTAI值
    如果CTAI > 85%，考虑高风险
    如果65% ≤ CTAI ≤ 85%，考虑中高风险
    如果CTAI < 65%，考虑低风险
3. 综合CAC和CTAI
    如果两者都高，风险最高
    如果一高一低，需要进一步考虑
    如果两者都低，风险相对较低
4. 考虑结节大小
    如果大小 ≥ 8mm，增加风险
    如果大小 < 8mm，相对风险较低

## 3. 建议生成

基于以上评估，请给出具体建议：

1. 治疗方案
    是否建议积极治疗（如手术）？
    是否建议密切随访？
    是否建议常规体检？
2. 随访计划
    建议多长时间进行一次复查？
    每次复查应包括哪些检查项目？
3. 其他注意事项
    是否需要考虑患者的其他个人因素？
    是否有其他需要患者注意的事项？

## 4. 总结

请总结你的评估过程和最终建议，确保逻辑清晰，建议具体可行。

请以JSON格式输出你的结果，包括以下字段：
"risk_level": "高/中高/中/中低/低",
"treatment_plan": "建议的治疗方案",
"follow_up_plan": "建议的随访计划",
"additional_notes": "其他注意事项",
"summary": "总结"
"""

In [ ]:
prompt = create_tot_prompt(10, '90%', 15)
messages=[
            {"role": "system", "content": "You are a medical AI assistant specialized in lung nodule risk assessment."},
            {"role": "user", "content": prompt}
        ]
predict(messages)

In [ ]:
def create_tot_prompt_2(CAC_VALUE, CTAI_VALUE,NODULE_SIZE, NODULE_CHANGE):
    return f"""
1. 初始化
请根据以下信息评估肺结节风险并给出建议：

CAC值: {CAC_VALUE}
CTAI值: {CTAI_VALUE}
结节大小: {NODULE_SIZE} mm
结节变化: {NODULE_CHANGE}

2. 风险评估
请按以下步骤进行思考，并在每一步详细解释您的推理过程：

评估CAC值:

CAC > 9: 高风险
3 ≤ CAC ≤ 9: 中高风险
CAC < 3: 低风险


评估CTAI值:

CTAI > 85%: 高风险
65% ≤ CTAI ≤ 85%: 中高风险
CTAI < 65%: 低风险


综合CAC和CTAI:

如果CAC > 9且CTAI > 85%，判定为最高风险
如果CAC > 9但CTAI < 85%，或3 ≤ CAC ≤ 9且CTAI > 85%，判定为高风险
其他情况需要进一步考虑结节大小和其他因素


考虑结节大小:

结节大小 ≥ 8mm: 显著增加风险
结节大小 < 8mm: 相对风险较低
结节大小 > 15mm: 即使其他指标较低也需要特别关注

考虑结节变化：
结节增大：考虑恶性病变，感染或炎症
结节变小：考虑良性病变
无明显变化：稳定的良性变化



3. 建议生成
基于以上评估，请给出具体建议，并解释每个建议的理由：

治疗方案:

是否建议积极治疗（如手术）？在什么条件下建议手术？
是否建议抗炎治疗？抗炎治疗多久后复查？
是否建议密切随访？频率如何？
是否建议常规体检？频率如何？


随访计划:

建议多长时间进行一次复查？为什么？
每次复查应包括哪些检查项目？为什么需要这些项目？


其他注意事项:

有哪些生活方式的建议？
患者需要注意哪些可能的症状变化？



4. 特殊情况考虑
请考虑以下特殊情况，并给出相应的建议：

如果患者拒绝手术，替代方案是什么？
如果结节在随访中发现变化（如大小增加、密度增高），应如何处理？
对于年龄较大或有其他健康问题的患者，评估标准是否需要调整？

5. 总结
请总结您的评估过程和最终建议，确保逻辑清晰，建议具体可行。包括：

总体风险评估
主要建议及其理由
需要进一步关注或澄清的问题

请以JSON格式输出您的结果，包括以下字段：
"risk_level": "高/中/低",
"cac_assessment": "CAC值评估结果",
"ctai_assessment": "CTAI值评估结果",
"nodule_size_assessment": "结节大小评估结果",
"overall_risk_assessment": "综合风险评估结果",|
"treatment_plan": "建议的治疗方案",
"follow_up_plan": "建议的随访计划",
"additional_notes": "其他注意事项",
"special_considerations": "特殊情况的考虑",
"summary": "总结"
"""

In [ ]:
prompt2 = create_tot_prompt_2(9, '30%', 9,"较前增大")
system_prompt = f"""
您是一位专门从事肺结节风险评估的AI医疗助手。您的知识基于最新的医学研究和指南。在评估过程中，请严格遵循以下原则：

1.使用CAC（Circulating genetically Abnormal Cell）和CTAI（Lung Nodule risk values from CTAI）作为主要评估指标。
2.考虑结节大小作为额外的风险因素。
3.提供基于证据的建议，并明确说明建议的依据。
4.在不确定的情况下，建议进一步检查或咨询专科医生。
5.所有建议都应考虑患者安全为首要原则。
"""
messages_2 = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt2}
]
predict(messages_2)